Hi all!
In this script, we use (deep) reinforcement learning to solve the knapsack problem.

What is the knapsack problem (KP)? 
The KP is a problem where you are given a knapsack (a bag) with limited capacity and a set of objets.
Objects have two attributes, prize and weight. Your goal is to select some objects such that you maximize the prize of the object chosen without exceeding the capacity of the knapsack.

For example, given a knapsack of capacity 1 (also in the following, we assume weights to be normalized, so capacity knapsack = 1) and objects [prize, weight] = {[3,0.8],[1,0.25],[1,0.25],[1,0.25],[1,0.25]} the best solution would be to NOT pick up $obj_{0}$ (even if it has the greatest prize) while picking up all the other objects.

Why I've chosen the KP? Because the KP is often considered the easiet among the NP-Hard problem (meaning that it takes exponential number of steps to achieve the optimal solution)

The ingredients of out RL will be the following:
- multi task RL (because every instance is different)
- DQN: deep Q-network
- value approxiamtion via deep NN. Since the solution does not depends on the order of the objects (and for many other reasons), we use the attention mechanism in the NN. Attention is permutation - indifferent.
- off policy. We use eps-greedy to explore but we use greedy to test.

The overall script will be devided in (more or less) five big modules for repetibility usage.
1. environment (generate tasks, new states, rewards,..)
2. replay memory (memorizes transitions)
3. Model (decides the exploration/evaluation policy)
4. Learner (updates the model. input: memory, model)
5. Runner (runs episodes. input: environment, model)

As always, we start by importing useful libraries

In [1]:
import matplotlib.pyplot as plt

import numpy as np 
import pandas as pd
import torch 
import random
import itertools
import copy
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle

%matplotlib inline

random.seed(1234)

import torch
if torch.cuda.is_available():
    print('GPU')

/home/big_bamboo/anaconda3/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Then we define the first big module: ENVIRONMENT
- the environment generates a task with num_objs (input) objects. 
- the enviroment can either generate a real instance (by default) or a fake one of which we know the optimal solution.
- the environment returns the objects and their target (which is computed via an heuristic). # CHANGE THIS

NOTE! for some aspects, what we call environment is actually more a task than an environment.

In [2]:
class ObjectCass:
    
    Prize = None
    Weight = None
    
    def __init__(self, reward, weight):
        
        self.Prize = reward
        self.Weight = weight

class StateClass:
    
    ObjectsFeatures = None
    res_capacity = None
    Chosen = None
    price = None
    Final = None
    IndexesActionsNotMasked = None
    
    def __init__(self, ObjectsFeatures, res_capacity, Chosen, price, Task):
        
        self.res_capacity = res_capacity
        self.Chosen = Chosen
        self.price = price
        self.Final = 0
        self.IndexesActionsNotMasked = self.StateMaskFunction(Task)
        if len(self.IndexesActionsNotMasked)==0:
            self.Final=1
        self.ObjectsFeatures = ObjectsFeatures
        # change the boolean feature of the not selectable actions
        for i in range(len(self.ObjectsFeatures)):
            if i not in self.IndexesActionsNotMasked:
                self.ObjectsFeatures[i][2]=0.0
        
        return

    def StateMaskFunction(self, Task):
        
        indexes = [i for i in range(len(Task.Objects)) 
                    if i not in self.Chosen and 
                    not (self.res_capacity - Task.Objects[i].Weight < 0)]
    
        return indexes 
    
class TaskClass:
    
    Objects = None
    total_capacity = None
    target = None
    
    def __init__(self, Objects, total_capacity):
        
        self.Objects = Objects
        self.total_capacity = total_capacity
        self.target = self.Heuristic(Objects)

    def Heuristic(self, Objects):
    
        Objects_sorted = sorted(Objects, key=lambda obj: -float(obj.Prize)/obj.Weight)
        weight_total = 0
        price_total = 0
        for obj in Objects_sorted:
            if weight_total+obj.Weight>1:
                continue
            price_total+=obj.Prize
            weight_total+=obj.Weight

        target = torch.tensor(price_total).float()
        target.requires_grad = False

        return target
    
    def initial_state(self):
        
        price = torch.tensor(0).float() #
        
        ObjectsFeatures = []
        for i in range(len(self.Objects)):
            ObjectsFeatures.append(torch.tensor([self.Objects[i].Prize, self.Objects[i].Weight, 1.0 , self.total_capacity]))
        ObjectsFeatures = torch.stack(ObjectsFeatures) 
        
        return StateClass(ObjectsFeatures, self.total_capacity, [], price, self)
    
    def step(self, state_old, action):
    
        capacity = state_old.res_capacity-self.Objects[action].Weight
        price = state_old.price+self.Objects[action].Prize
        New_Chosen = copy.copy(state_old.Chosen)
        New_Chosen.append(action)
        ObjectsFeatures = state_old.ObjectsFeatures.clone()
        for i in range(len(ObjectsFeatures)):
            ObjectsFeatures[i][-1] = capacity
        # the boolens of the unchosable objects will be set to zero with the following command
        state_new = StateClass(ObjectsFeatures, capacity, New_Chosen, price, self)
        
        # new state , reward
        return state_new, self.Objects[action].Prize

class EnvironmentClass:
    
    num_objs = None
    
    def __init__(self, num_objs):
        
        self.num_objs = num_objs
        
        return
    
    def NewTask(self, FakeBool = False, prize_min=0.01, prize_max=1, weight_min=0.01, weight_max=1, shuffle = False):
  
        if FakeBool:
            # fake task
            # best solution = [1,0,1,0,1,0,1,0,1,0,1]
            Objects = []
            for i in range(self.num_objs):
                if i%2==1:
                    prize = 0.1
                    weight = 0.99
                    obj = ObjectCass(prize,weight)
                    Objects.append(obj)
                else:
                    prize = 1
                    weight = (2/float(num_objs))-0.001
                    obj = ObjectCass(prize,weight)
                    Objects.append(obj)
            if shuffle:
                random.shuffle(Objects)

        else:
            Objects = []
            for i in range(self.num_objs):
                prize = round(random.uniform(prize_min, prize_max), 2)
                weight = round(random.uniform(weight_min, weight_max), 2)
                obj = ObjectCass(prize,weight)
                Objects.append(obj)

        capacity = torch.tensor([1]).float() #

        self.Objects = Objects
        self.capacity = capacity
            
        return TaskClass(Objects, capacity)   
    
    def InitialStateNewTask(self, FakeBool = False, prize_min=0.01, prize_max=1, weight_min=0.01, weight_max=1, shuffle = False):
        
        Task = self.NewTask(FakeBool = False, prize_min=0.01, prize_max=1, weight_min=0.01, weight_max=1, shuffle = False)
        initial_state = Task.initial_state()
        
        return initial_state, Task
    

env = EnvironmentClass(2)
s_0, task = env.InitialStateNewTask()
s_1, reward = task.step(s_0,0)
print('s_0 cap', s_0.res_capacity)
print('s_1 cap', s_1.res_capacity)
print('s_0 price', s_0.price)
print('s_1 price', s_1.price)
print('s_0 obj features', s_0.ObjectsFeatures)
print('s_1 obj features', s_1.ObjectsFeatures)
print('reward', reward)

s_0 cap tensor([1.])
s_1 cap tensor([0.5500])
s_0 price tensor(0.)
s_1 price tensor(0.9700)
s_0 obj features tensor([[0.9700, 0.4500, 1.0000, 1.0000],
        [0.0200, 0.9100, 1.0000, 1.0000]])
s_1 obj features tensor([[0.9700, 0.4500, 0.0000, 0.5500],
        [0.0200, 0.9100, 0.0000, 0.5500]])
reward 0.97


Implementation based on: https://towardsdatascience.com/understanding-actor-critic-methods-931b97b6df3f

Second big module: Model
- the model contains the training/evaluation policy and it is responsable for choosing the actions
- it contains the NN used for value function approximation

In [3]:
class Net_Attention(torch.nn.Module):
    
    def __init__(self, num_input_features,h_model, num_head ,num_layers, dim_feedforward, p_dropout, num_outputs):    
        
        super(Net_Attention, self).__init__()
        
        self.emb = torch.nn.Linear(num_input_features,  h_model)
        encoder_layer = torch.nn.TransformerEncoderLayer(d_model=h_model, nhead=num_head, 
                                                   dim_feedforward=dim_feedforward, dropout=p_dropout)
        self.transformer_encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
                
        # FINAL LAYER
        self.final_layer_0 = torch.nn.Linear(h_model,h_model)
        self.final_layer_1 = torch.nn.Linear(h_model,1)

    def forward(self, States, no_grad = False):
        
        ObjectsFeatures = [state.ObjectsFeatures for state in States]
        ObjectsFeatures = torch.stack(ObjectsFeatures)
        
        # define relu operation
        ReLU = torch.nn.ReLU()
        
        if no_grad: #doesn't compute the gradients (hence no back propagation, but faster). 
            with torch.no_grad(): # to be used only in evaluations and target (not in training the prediction)        
                E = self.emb(torch.transpose(ObjectsFeatures,0,1))
                # where 
                # S is the source sequence length,  
                # N is the batch size,  
                # E is the feature number        
                # you want 
                # E.size() = S,N,E = num_obj x batch x h_model
                c = self.transformer_encoder(E)
                c = self.final_layer_0(c)
                c = ReLU(c)
                Qvals = self.final_layer_1(c)
                Qvals = torch.squeeze(Qvals, dim = 2)
                Qvals = torch.transpose(Qvals,0,1)
                
                return Qvals
            
        E = self.emb(torch.transpose(ObjectsFeatures,0,1))
        # where 
        # S is the source sequence length,  
        # N is the batch size,  
        # E is the feature number        
        # you want 
        # E.size() = S,N,E = num_obj x batch x h_model
        c = self.transformer_encoder(E)
        c = self.final_layer_0(c)
        c = ReLU(c)
        Qvals = self.final_layer_1(c)
        Qvals = torch.squeeze(Qvals, dim = 2)
        Qvals = torch.transpose(Qvals,0,1)
                
        return Qvals

class ModelClass():

    def __init__(self, Net_prediction, Net_target, training_policy, evaluation_policy, epsilon_max, epsilon_min):
        
        self.Net_prediction = Net_prediction
        self.Net_target     = Net_target
        if training_policy!='eps-greedy' or evaluation_policy!='greedy':
            print('at least one of this exploration/evaluation policy are still to implement')
            error
        else:
            self.epsilon_min = epsilon_min
            self.epsilon_max = epsilon_max
        self.training_policy   = training_policy
        self.evaluation_policy = evaluation_policy

    def Greedy(self, Qvals, state):
        
        Qvals_selectable = [Qvals[i] for i in range(len(Qvals)) if i in state.IndexesActionsNotMasked]
        Qvals_selectable = torch.stack(Qvals_selectable)
        action = torch.argmax(Qvals_selectable)
        action_index = state.IndexesActionsNotMasked[action]

        return action_index
    
    def EpsGreedy(self, Qvals,state, iteration_counter, num_iterations):
    
        # select action
        epsilon = (epsilon_max-epsilon_min)*(num_iterations-iteration_counter)/num_iterations + epsilon_min
        if random.random() < epsilon:
            action_index = random.choice(state.IndexesActionsNotMasked)
        else:
            action_index = self.Greedy(Qvals,state)

        return action_index
    
    def training_step(self, state, iteration_counter, num_iterations):
                
        self.Net_prediction.train()
        self.Net_target.train()
        if self.training_policy == 'eps-greedy':
            Qvals = self.Net_prediction([state])
            Qvals = Qvals.squeeze()
            return self.EpsGreedy(Qvals, state, iteration_counter, num_iterations)
        error
    
    def evaluation_step(self, state):
        
        self.Net_prediction.eval()
        if self.evaluation_policy == 'greedy':
            Qvals = self.Net_prediction([state], no_grad = True)
            Qvals = Qvals.squeeze()
            return self.Greedy(Qvals, state)
        error

Now we have a function that, given a state, it returns the Q values. We have to define the loss 

Third module: BUFFER (aka replay memory)
- the buffer contains the last #replay_lenght transictions.
- each experience in the buffer is as a tuple <$s_{old}$, action, r, $s_{new}$>

In [4]:
class ReplayMemoryClass:
    
    replay_lenght = None
    minibatch_size = None
    Buffer = None
    
    def __init__(self,replay_lenght, minibatch_size):
        
        self.replay_lenght = replay_lenght
        self.minibatch_size = minibatch_size
        self.Buffer = []
        
    def Add(self, Transitions):
                
        while len(self.Buffer)+len(Transitions)>=self.replay_lenght:
            self.Buffer.remove(random.choice(self.Buffer))
        for t in Transitions:
            self.Buffer.append(t)
        
        return
        
    def Minibatch(self, new_transactions = 0):
        
        if len(self.Buffer) <= self.minibatch_size:
            Elements = self.Buffer
        else:
            new_transactions = min(new_transactions, self.minibatch_size)
            last_transactions = self.Buffer[-new_transactions:]
            Elements = random.sample(self.Buffer, self.minibatch_size - new_transactions)
            if self.minibatch_size - new_transactions==0:
                print('check that everything is fine and dandy')
                print(len(last_transactions))
                print(len(Elements))
                stoptocheck
            Elements = last_transactions + Elements
            Minibatch = Elements
        
        # CHANGE THE WAY YOU SAVE THINFS IN THE BUFFER!!!!
        # SAVE DIRECTLY AS THE LISTS BELOW
        
        States = [experience[0] for experience in Elements]
        Actions = [experience[1] for experience in Elements]
        Rewards = [experience[2] for experience in Elements]
        New_States = [experience[3] for experience in Elements]
        
        Minibatch = [States, Actions, Rewards, New_States]
        
        return Minibatch

Fourth module: LEARNER
- receives minibatches
- updates the Model

In [5]:
class LearnerClass():

    Model = None
    optimizer = None
    gamma = None
    tau = None
    
    def __init__(self,Model, optimizer, tau, gamma = 1):
        
        self.Model = Model
        self.optimizer = optimizer
        self.tau = tau
        self.gamma = gamma

    def Learning_step(self, Minibatch):
    
        # zero the gradient buffers
        self.optimizer.zero_grad() 

        Losses = []
        States, Actions, Rewards, New_States = Minibatch

        New_States_Target = [New_States[i] for i in range(len(New_States)) if New_States[i].Final==0]
        if len(New_States_Target)>0:
            Max_New_States_Target, _ = torch.max(Model.Net_target(New_States_Target, no_grad = True), dim = 1)
        else:
            Max_New_States_Target = []
        Max_New_States = []
        j = 0
        for i in range(len(New_States)):
            if New_States[i].Final==0: # not final
                Max_New_States.append(Max_New_States_Target[j])
                j+=1
            else:
                zero_tensor_scalar = torch.tensor(0).float()
                zero_tensor_scalar.requires_grad = False
                Max_New_States.append(zero_tensor_scalar)
        Max_New_States = torch.stack(Max_New_States)
        Rewards = torch.tensor(Rewards)
        New_States_Prediction = self.Model.Net_prediction(States)
        Predictions_Actions = [New_States_Prediction[i][Actions[i]] for i in range(len(New_States_Prediction))]
        Predictions_Actions = torch.stack(Predictions_Actions)

        Losses = (Rewards + self.gamma*Max_New_States - Predictions_Actions)**2 

        loss = Losses.mean()
        loss.backward()

        # gradient clipping
        clip_value = 1
        for p in self.Model.Net_prediction.parameters():
            p.register_hook(lambda grad: torch.clamp(grad, -clip_value, clip_value))

        # apply gradient
        self.optimizer.step()

        # soft target update
        for param_target, param_predict in zip(self.Model.Net_target.parameters(), self.Model.Net_prediction.parameters()):
            param_target.data.copy_(param_target.data * (1.0 - self.tau) + param_predict.data * self.tau)        
        
        return loss.tolist()

Fifth module: RUNNER
- takes the environment and the module
- runs a transaction. Or should it run an episode?

In [6]:
class RunnerClass():

    Environment = None
    Model = None
    
    def __init__(self, Environment, Model):
        
        self.Model = Model
        self.Environment = Environment
        
    def NewTrainingTask(self, iteration_counter,num_iterations):
        
        Transitions = []
        state, task = self.Environment.InitialStateNewTask()
        while state.Final == 0:
            action_index = self.Model.training_step(state, iteration_counter,num_iterations)
            new_state, reward = task.step(state, action_index)
            Transitions.append([state, action_index, reward, new_state])
            state = new_state
         
        return Transitions, task.target
    
    def NewEvaluationTask(self):
        
        state, task = self.Environment.InitialStateNewTask()
        G = 0
        while state.Final == 0:
            action_index = self.Model.evaluation_step(state)
            new_state, reward = task.step(state, action_index)
            G+=reward
            state = new_state
         
        return G, task.target

Now we create a function to evaluate the trained network

In [7]:
def EvaluatePrint(R,T, PATH_Final, time_counter):
    
    R_norm = [R[i]/T[i] for i in range(len(R))]
    T_norm = [1 for i in range(len(R))]
        
    if PATH_Final!=False:
        plt.figure()
        plt.plot(R)
        plt.plot(R, label='Rewards')
        plt.plot(T)
        plt.plot(T, label='heuristic')
        plt.ylabel(' Rewards ' )
        plt.legend()
        plt.ylim(bottom=0)

        plt.savefig(PATH_Final+'Evaluation_Rewards_num_'+str(time_counter)+'.png', bbox_inches='tight')
        
        plt.figure()
        plt.plot(R_norm)
        plt.plot(R_norm, label='Rewards (normalized)')
        plt.plot(T_norm)
        plt.plot(T_norm, label='heuristic (normalized)')
        plt.ylabel(' Rewards (normalized)' )
        plt.legend()
        plt.ylim(bottom=0)
        
        plt.savefig(PATH_Final+'Evaluation_Rewards_Normalized_num_'+str(time_counter)+'.png', bbox_inches='tight')


    print('average norm reward (net)   : ', np.mean(R_norm))
    if np.mean(R_norm)>np.mean(T_norm):
        return 1
    return 0

def Evaluate(num_iterations, Runner, times = 1, PATH_Final = False, examples = 0):
    
    better = 0
    for time_counter in range(1,times+1):
        print('evaluating model..('+str(time_counter)+' out of '+str(times)+')')
        G_tot = []
        heuristic_tot = []
        for iteration_counter in range(num_iterations):
            # perform new task
            G, heuristic = Runner.NewEvaluationTask()
            G_tot.append(G)
            heuristic_tot.append(heuristic)
        b = EvaluatePrint(G_tot,heuristic_tot, PATH_Final, time_counter)
        better+=b
    print('\n\n\n')
    print('the NN was better than the heuristic '+str(better)+' times out of '+str(times))                    

Now we create our 5 modules (or load them from existing ones)

In [8]:
# seed
random.seed(1234)
# model dimensions
num_iterations = 4*2500
num_objs = 10
num_input_features = 4 # prize, weight, res_capacity, Bool_alreday_chosen
num_outputs = num_objs
h_model = 16
num_head = 4
num_layers = 2
dim_feedforward = 16
p_dropout = 0.0
epsilon_max = 1
epsilon_min = 0.05
replay_lenght = 10e6
minibatch_size = 32
learning_rate = 1e-4
momentum = 0.9
tau = 0.005
PATH = '/home/big_bamboo/RL for the KP/'
PATH_Aux = '/home/big_bamboo/RL for the KP/Saved_objects_Aux/'
PATH_Final = '/home/big_bamboo/RL for the KP/Final_Objects/'
num_iterations_start = 0
Q_print = []

# is it a new test?
LoadTest = False
# objects in the middel of training (Aux) or completely computed (Final)
Path_load = PATH_Aux
if LoadTest:
    #Net_predict.load_state_dict(torch.load(Path_load+'Net_Predict'))
    #Net_target.load_state_dict(torch.load(Path_load+'Net_Target'))
    with open(Path_load+'RLKP-num_iterations_start-Save.pkl', 'rb') as input:
        num_iterations_start = pickle.load(input)
    with open(Path_load+'RLKP-Learner-Save.pkl', 'rb') as input:
        Learner = pickle.load(input)
    with open(Path_load+'RLKP-Runner-Save.pkl', 'rb') as input:
        Runner = pickle.load(input)
    env = Runner.Environment
    Model = Learner.Model
    Runner.Model = Model
    with open(Path_load+'RLKP-ReplayMemory-Save.pkl', 'rb') as input:
        ReplayMemory = pickle.load(input)
    with open(Path_load+'RLKP-Q_print-Save.pkl', 'rb') as input:
        Q_print = pickle.load(input)
else:
    # environment
    env = EnvironmentClass(num_objs)
    # model
    Net_predict = Net_Attention(num_input_features,h_model, num_head ,num_layers, dim_feedforward, p_dropout, num_outputs)
    Net_target = Net_Attention(num_input_features,h_model, num_head ,num_layers, dim_feedforward, p_dropout, num_outputs)
    Model = ModelClass(Net_predict, Net_target, 'eps-greedy', 'greedy', epsilon_max, epsilon_min)
    # Replay Memory
    ReplayMemory = ReplayMemoryClass(replay_lenght, minibatch_size)
    # Learner
    optimizer = torch.optim.SGD(Net_predict.parameters(), lr=learning_rate, momentum = momentum)
    Learner = LearnerClass(Model, optimizer, tau)
    # Runner
    Runner = RunnerClass(env, Model)

def SaveData(iteration_counter,Learner,Runner,ReplayMemory,Q_print, PATH_save):
    
    #torch.save(Net_predict.state_dict(), PATH+'Net_Predict')
    #torch.save(Net_target.state_dict(), PATH+'Net_Target')
    with open(PATH_save+'RLKP-num_iterations_start-Save.pkl', 'wb') as output:
        pickle.dump(iteration_counter, output, pickle.HIGHEST_PROTOCOL)
    with open(PATH_save+'RLKP-Learner-Save.pkl', 'wb') as output:
        pickle.dump(Learner, output, pickle.HIGHEST_PROTOCOL)
    with open(PATH_save+'RLKP-Runner-Save.pkl', 'wb') as output:
        pickle.dump(Runner, output, pickle.HIGHEST_PROTOCOL)
    with open(PATH_save+'RLKP-ReplayMemory-Save.pkl', 'wb') as output:
        pickle.dump(ReplayMemory, output, pickle.HIGHEST_PROTOCOL)
    with open(PATH_save+'RLKP-Q_print-Save.pkl', 'wb') as output:
        pickle.dump(Q_print, output, pickle.HIGHEST_PROTOCOL)
    
    return

and we also define a function to print the initial Q vales.
(this function is not really smart, it recomputes the Q values. think of another way to do it)

In [9]:
def MaxInitialQvalues(Transitions, Net_predict, target_heuristic):
    
    first_state, action_index, reward, new_state = Transitions[0]
    Qvals = Net_predict([first_state], no_grad = True)
    Qvals = Qvals.squeeze().tolist()
    
    return [Qvals, target_heuristic]

def PrintMaxQval(Q_prints_tuple, PATH_Final):
    
    Q_prints = [Q[0] for Q in Q_prints_tuple]
    T = [Q[1] for Q in Q_prints_tuple]
    
    Q_max = [max(Q_vals) for Q_vals in Q_prints]
    plt.figure()
    for i in range(len(Q_prints[0])):
        Q_Action_i = [Q_prints[j][i] for j in range(len(Q_prints))]
        #plt.plot(Q_Action_i)
        plt.plot(Q_Action_i, label='Q_Action_'+str(i))
    plt.plot(Q_max, label='Q_max')
    plt.legend()
    plt.savefig(PATH_Final+'Q_Vals_Training.png', bbox_inches='tight')
    
    # same but normalized
    Q_max = [Q_max[i]/T[i] for i in range(len(Q_max))]
    plt.figure()
    for i in range(len(Q_prints[0])):
        Q_Action_i = [Q_prints[j][i]/T[j] for j in range(len(Q_prints))]
        #plt.plot(Q_Action_i)
        plt.plot(Q_Action_i, label='Q_Action_Norm_'+str(i))
    plt.plot(Q_max, label='Q_Max_Norm')
    Ones = [1 for j in range(len(Q_prints))]
    plt.plot(Ones)
    plt.legend()
    plt.savefig(PATH_Final+'Q_Vals_Training_Norm.png', bbox_inches='tight')
    
    return

And now we use them.

In [ ]:
for iteration_counter in range(num_iterations_start, num_iterations):
    if iteration_counter%100==0:
        SaveData(iteration_counter,Learner,Runner,ReplayMemory,Q_print, PATH_Aux)
        print('task number: ',(iteration_counter))
    # perform new task
    Transitions, target_heuristic = Runner.NewTrainingTask(iteration_counter, num_iterations)
    Q_print.append(MaxInitialQvalues(Transitions, Model.Net_prediction, target_heuristic))
    # save transition
    ReplayMemory.Add(Transitions)
    # learn        
    if iteration_counter>= ReplayMemory.minibatch_size:
        Learner.Learning_step(ReplayMemory.Minibatch())

SaveData(num_iterations,Learner,Runner,ReplayMemory,Q_print, PATH_Final)
PrintMaxQval(Q_print, PATH_Final)
print('evaluation: ')
Evaluate(100, Runner, PATH_Final = PATH_Final, times = 10)

task number:  0
task number:  100
task number:  200
task number:  300
task number:  400
task number:  500
task number:  600
task number:  700
task number:  800
task number:  900
task number:  1000
task number:  1100
task number:  1200
task number:  1300
task number:  1400
task number:  1500
task number:  1600
task number:  1700
task number:  1800
task number:  1900
task number:  2000
task number:  2100
task number:  2200
task number:  2300
task number:  2400
task number:  2500
task number:  2600
task number:  2700
task number:  2800
task number:  2900
task number:  3000
task number:  3100
task number:  3200
task number:  3300
task number:  3400
task number:  3500
task number:  3600
task number:  3700
task number:  3800
task number:  3900


In [ ]:
# is it a new test?
LoadTest = True
# objects in the middel of training (Aux) or completely computed (Final)
Path_load = PATH_Aux
Path_load = PATH_Final
if LoadTest:
    #Net_predict.load_state_dict(torch.load(Path_load+'Net_Predict'))
    #Net_target.load_state_dict(torch.load(Path_load+'Net_Target'))
    with open(Path_load+'RLKP-num_iterations_start-Save.pkl', 'rb') as input:
        num_iterations_start = pickle.load(input)
    with open(Path_load+'RLKP-Learner-Save.pkl', 'rb') as input:
        Learner = pickle.load(input)
    with open(Path_load+'RLKP-Runner-Save.pkl', 'rb') as input:
        Runner = pickle.load(input)
    env = Runner.Environment
    Model = Learner.Model
    Runner.Model = Model
    with open(Path_load+'RLKP-ReplayMemory-Save.pkl', 'rb') as input:
        ReplayMemory = pickle.load(input)
    with open(Path_load+'RLKP-Q_print-Save.pkl', 'rb') as input:
        Q_print = pickle.load(input)
Evaluate(100, Runner, times = 10)

TO DO:
- check if it works
- prioritized replay is actually working.

ALSO:
- lr scheduler
- should episodes been created in batches? If so, how to 'schedlue' them in the replay buffer? (The idea is to pick the last one in the replay, but how to do this if you generate X episodes at a time?)

In [ ]:
# set to train

# optimizer scheduler
#lmbda = lambda epoch: 1
#scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)

#if (e_counter)%int(num_epoch/5)==0 and e_counter>minibatch_size:
#    scheduler.step()
#    for param_group in optimizer.param_groups:
#        print('learning rate ', param_group['lr'])
#        if param_group['lr']<1e-10:
#            print('the lr dropped too low, something is wrong')
#            error